In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
from keras.layers import  Dense, Flatten
from glob import glob
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning)

from keras.applications import MobileNet, VGG16, VGG19, ResNet50,\
      ResNet101, EfficientNetB3, DenseNet201, ResNet152

from keras.callbacks import ModelCheckpoint

In [ ]:
#data path
train_data_path = 'ALL TVT/train'
validation_data_path = 'ALL TVT/val'
test_data_path = 'ALL TVT/test'

In [ ]:
IMAGE_SIZE = [224, 224] #size of image

In [ ]:
folders = glob('ALL TVT/train/*')
folders

In [ ]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_batchsize = 32
val_batchsize = 32
img_width, img_height = IMAGE_SIZE[0], IMAGE_SIZE[1]

train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(img_width, img_height),
        batch_size=train_batchsize,
        class_mode='categorical',
        shuffle=False)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_path,
        target_size=(img_width, img_height),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

n_epochs = 10 

models_dict = {
       'MobileNet': MobileNet,
        'VGG16': VGG16,
        'VGG19': VGG19,
        'ResNet50': ResNet50,
        'Resnet101': ResNet101,
        'EfficientNetB3': EfficientNetB3,
        'DenseNet201': DenseNet201,
        'Resnet152': ResNet152
    } #models used

In [ ]:
with tf.device(device):
    for model_name in models_dict:
        print(f"Training {model_name}")
        print('-----------------------------------')
        KerasModel = models_dict[model_name]
        base_model = KerasModel(input_shape=IMAGE_SIZE + [3], 
                                weights='imagenet', 
                                include_top=False)
        for layer in base_model.layers:
            layer.trainable = False

        x = Flatten()(base_model.output)
        prediction = Dense(len(folders), activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=prediction)
        model.summary()

        adam = tf.optimizers.Adam()
        model.compile(loss='categorical_crossentropy',
                    optimizer=adam,
                    metrics=['accuracy'])
        
        model_checkpoint = ModelCheckpoint(f'{model_name}.h5', 
                                           monitor='val_loss', 
                                           save_best_only=True)
        callbacks = [model_checkpoint]
        history = model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples/train_generator.batch_size ,
            epochs=n_epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples/validation_generator.batch_size,
            verbose=1,
            callbacks=callbacks)

        # Plot accuracy
        fig1 = plt.figure(figsize=(12,6))
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.ylim(0.7,1)
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epochs')
        plt.legend(['train', 'validation'])
        plt.savefig(f'{model_name}_Accuracy.png')
        plt.show()

        # Plot loss
        fig2 = plt.figure(figsize=(12,6))
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.legend(['train', 'validation'])
        plt.savefig(f'{model_name}_Loss.png', bbox_inches='tight')
        plt.show()